# Projeto 2 - Ciência dos Dados

Nome: Carlos Eduardo Dip

Nome: Gianluca Lazzaris Giudici

# Proposta
___

Neste projeto, a proposta é criar um identificador de sentimento, a partir de um analisador Naïve-Bayes, que faz uso principalmente do Teorema de Bayes, e é um modelo de Machine Learning.
___
# Classificador automático de sentimento

<img src = 'logo.jpg' style = 'width:30%'>

Empresa escolhida: Uber

___
## Preparando o ambiente no jupyter:

Primeiro importamos os módulos que serão utilizados:

In [56]:
import numpy as np
import os.path
import pandas as pd
import json
import re
import string
import functools
import operator
from random import shuffle
import matplotlib.pyplot as plt

try:
    import emoji
except:
    !pip install emoji --upgrade
    import emoji

___
### Coleta de Dados:

Para a análise que será feita, serão usados tweets (da plataforma Twitter), que serão extraídos através de uma API chamada Tweepy;
Os dados foram coletados em outro notebook, chamado Coletor.ipynb. Abaixo, será importado o excel com os dados já classificados.

___

## Limpeza, aquisição e organização dos dados

Primeiro, construímos dataframes (da biblioteca Pandas), para facilitar a visualização e utilização dos dados.

In [2]:
data_training = pd.read_excel('data/Uber.xlsx')
data_validation = pd.read_excel('data/Uber.xlsx', 'Teste')

data_training_R = data_training.loc[data_training.Classificacao == 1]
data_training_NR = data_training.loc[data_training.Classificacao == 0]
TrainingString_Relevant = ''
TrainingString_NotRelevant = ''

Agora, podemos visualizar o formato que o dataframe se encontra. Ele possuí 2 colunas (fora o índice), uma delas indica qual a classificação (1 é relevante, e 0 é irrelevante), e a outra mostra o tweet que foi avaliado.


In [3]:
data_training.head(10)

,Treinamento,Classificacao
0,uma corrida com esse uber a única forma de pag...,0
1,uber testa em são paulo programa que avalia qu...,1
2,"vcs que andam de uber nem falem comigo, além d...",0
3,a fic: terminei o último episódio de glee (dnv...,1
4,"olivares, cida \nsobre seu uber \n\nmas podia ...",0
5,"nossa que vontade de sentar no uber, quer dize...",0
6,adoro ler os feedbacks do uber https://t.co/lp...,1
7,uber com balinha tudo pra mim,1
8,so uber e faço isso todo dia... https://t.co/e...,1
9,moço do uber: vc é a melhor da classe? \neu: n...,1


Podemos perceber, contudo, que esses tweets possuem pontuação, emojis, e alguns caractéres como **\n**, que irão poluir a análise.

Por isso, produzimos algumas funções que são capazes de limpar os tweets, para facilitar a análise.

In [5]:
def check_laughter(word):
    for letter in word:
        if letter != 'k':
            return False
    return True

def cleanup(text):

    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text_split = ' '.join(word for word in text_split)
    
    text = ' '.join(word for word in text_split.split() if not word.startswith('https'))
    
    
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = text_subbed.lower()
    text_out = ' '
    
    
    for word in text_subbed.split():
        if check_laughter(word):
            text_out += ' haha'
        else:
            text_out += ' ' + word
    
    
    return text_out    

In [6]:
### --- Cleanup function demo --- ###
demo_string_ =  """
                X gon give it to ya
                Fuck wait for you to get it on your own
                X gon deliver to ya
                Knock knock, open up the door, it's real
                Wit the non-stop, pop pop and stainless steel
                Go hard gettin busy wit it
                https://img.imgur/1337, uber blah blah kkk, 👨🏿😷😷
                uber blah blah kkkkkkk https://img.imgur/1337, 👨🏿😷😷
                """

print(cleanup(demo_string_))
print(cleanup(demo_string_).split())

  x gon give it to ya fuck wait for you to get it on your own x gon deliver to ya knock knock open up the door it s real wit the non stop pop pop and stainless steel go hard gettin busy wit it uber blah blah haha 👨🏿 😷 😷 uber blah blah haha 👨🏿 😷 😷
['x', 'gon', 'give', 'it', 'to', 'ya', 'fuck', 'wait', 'for', 'you', 'to', 'get', 'it', 'on', 'your', 'own', 'x', 'gon', 'deliver', 'to', 'ya', 'knock', 'knock', 'open', 'up', 'the', 'door', 'it', 's', 'real', 'wit', 'the', 'non', 'stop', 'pop', 'pop', 'and', 'stainless', 'steel', 'go', 'hard', 'gettin', 'busy', 'wit', 'it', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷', 'uber', 'blah', 'blah', 'haha', '👨🏿', '😷', '😷']


Acima, temos um exemplo de uso da função que limpa os textos necessários. Ela é capaz de remover **\n** (caractéres de quebra de linha), hyperlinks (haviam vários, que não significavam nada), pontuações, e também separa emojis das palavras e entre si. Outra coisa que será útil para a análise é transformar risadas (comumente representadas por uma série de caractéres *k* seguidos), em uma representação padrão, que foi escolhida como 'haha'.

In [7]:
for phrase in data_training_R.Treinamento:
    TrainingString_Relevant += phrase
for phrase in data_training_NR.Treinamento:
    TrainingString_NotRelevant += phrase    
    
cleanup(TrainingString_NotRelevant)
cleanup(TrainingString_Relevant)

TrainingSeries_Relevant = pd.Series(TrainingString_Relevant.split()).value_counts(True)
TrainingSeries_NotRelevant = pd.Series(TrainingString_NotRelevant.split()).value_counts(True)


TSR_Absolute = pd.Series(TrainingString_Relevant.split()).value_counts()
TSNR_Absolute = pd.Series(TrainingString_NotRelevant.split()).value_counts()

SampleSize_Relevant = len(TSR_Absolute)
SampleSize_NotRelevant = len(TSNR_Absolute)
SampleSize_Total = SampleSize_Relevant+SampleSize_NotRelevant

TrainingSeries_Join = pd.Series((TrainingString_Relevant+TrainingString_NotRelevant).split()).value_counts(True)

___
### Montando o Classificador Naive-Bayes


In [70]:
P_Relevant = SampleSize_Relevant/SampleSize_Total
P_NotRelevant = SampleSize_NotRelevant/SampleSize_Total

In [72]:
### Classificador:

def evaluate_relevance(tweet = "", alpha = alpha, V = V):
    
    ## -- Cleaning tweet
    text = cleanup(tweet)
    
    ## -- Evaluates likelihood
    sumR = 0
    sumNR = 0
    T_r = (SampleSize_Relevant+alpha*V)
    T_nr = (SampleSize_NotRelevant+alpha*V)
    
    for word in text.split():
        
        if word in TrainingSeries_Relevant: 
            sumR += np.log((TSR_Absolute[word]+alpha)/T_r)
        else:
            sumR += np.log(alpha/T_r)
        if word in TrainingSeries_NotRelevant: 
            sumNR += np.log((TSNR_Absolute[word]+alpha)/T_nr)
        else:
            sumNR += np.log(alpha/T_nr)
    
    return sumR>sumNR


___
### Verificando a performance

Agora, usamos a função evaluate_relevance

In [64]:
evaluations = [int(evaluate_relevance(tweet, 1, 1250)) for tweet in data_validation.Teste]

data_validation['bayes'] = evaluations

Accuracy = {'True-True':0,
        'True-False':0,
        'False-True':0,
        'False-False':0
       }


for tweet,evaluation in zip(data_validation.classificacao, data_validation.bayes):
    if tweet and evaluation:
        Accuracy['True-True'] += 1
    elif tweet and not evaluation:
        Accuracy['True-False'] += 1
    elif not tweet and evaluation:
        Accuracy['False-True'] += 1
    elif not tweet and not evaluation:
        Accuracy['False-False'] += 1

Accuracy_Normalized = {}
S = sum(Accuracy.values())
for k,v in Accuracy.items():
    Accuracy_Normalized[k] = v/S
Accuracy_Normalized['Final-Accuracy'] = str(round((Accuracy_Normalized['True-True']+Accuracy_Normalized['False-False'])*100, 3)) + '%'
lista.append(Accuracy_Normalized['Final-Accuracy'])
a = data_validation.classificacao.value_counts(True)
print("Percebe-se então, que, da nossa validação {1}% são relevantes e {0}% não, de acordo com nossa avaliação manual.".format(round(a[0]*100,2),round(a[1]*100, 2)))
Accuracy_Normalized

Percebe-se então, que, da nossa validação 52.26% são relevantes e 47.74% não, de acordo com nossa avaliação manual.


{'True-True': 0.41708542713567837,
 'True-False': 0.10552763819095477,
 'False-True': 0.36180904522613067,
 'False-False': 0.11557788944723618,
 'Final-Accuracy': '53.266%'}

In [69]:
### Usando SKLearn:

S = data_training
S_teste = data_validation

Xt_train = S['Treinamento']
y_train = S['Classificacao'] == 1

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

count = CountVectorizer()
X_train = count.fit_transform(Xt_train)

model = MultinomialNB(alpha=0.5)
model.fit(X_train, y_train)

Xt_test = S_teste['Teste']
y_test = S_teste['classificacao'] == 1

X_test = count.transform(Xt_test)

y_pred = model.predict(X_test)

"A acurácia ideal seria: {0}%".format(round(accuracy_score(y_test, y_pred)*100,3))


'A acurácia ideal seria: 58.794%'

___
### Concluindo

### Reflexão sobre os resultados

A partir da análise dos dados, podemos concluir que nosso classificador Naïve-Bayes é péssimo, pois ele só é melhor do que presumir que tudo é relevante, por 1%. Alguns motivos para explicar por que isso ocorreu seriam; A baixa quantidade de tweets de treinamento (400tweets), a alta subjetividade do tema (difícil dizer se um tweet é positivo ou negativo, muitas vezes por sarcasmo, por exemplo), ou ainda, porque não separamos palavras em grupos semânticos (como flexões de um mesmo verbo).

___

### Outros usos do classificador Naïve-Bayes

Esse tipo de classificador tem vários usos. Muitos fenômenos podem ser previsto com enorme acurácia, [como por exemplo detecção de spam, categorização de notícias, reconhecimento facial em fotos, diagnóstico médico, reconhecimento de dígitos, precisão do tempo, (essencialmente qualquer cenário, onde a hipótese que os eventos são independentes entre si seja razoável)](https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used) etc.

___

### Por quê não treinar o modelo com ele mesmo, ou outro modelo automatizado

Não faria sentido treinar o modelo com dados que o próprio modelo produziu, já que assim, ele só iria fixar víeses e erros que já cometia antes. Treiná-lo com outro modelo seria possível, porém não seria tão simples, e envolveria conhecimentos sobre outras técnicas de machine learning, como aprendizado adversarial ou cooperativo. 

___

### Sobre o tratamento de palavras

Existem múltiplas técnicas de limpar textos, como o *stemming*, que poderiam ajudar muito a análise. Neste projeto, apenas removemos caracéteres como **\n**, que não possuem significado, assim como *hyperlinks* desnecessários (como ***https:/t.co.11613613.com***), e também fazemos algo parecido com o stemming para risadas, transformando qualquer string composta somente por ***k***'s, em ***haha***. Também separamos adequadamente os emojis, permitindo que sejam analisados como elementos individuais.



___
___

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis [OK]()
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis [OK]()
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação [OK]()
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento [OK]()
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto [OK]()
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa) [OK]()
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) **Explicação da técnica usada para lidar com palavras novas**

[Other applications of Naïve-Bayes](https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used) **Exemplos de uso do classificador em outros contextos**

[Explanation fo beta-smoothing and calculating the alpha and beta coefficients](https://stats.stackexchange.com/questions/47916/bayesian-batting-average-prior/47921#47921) **Outro smoothing, que não foi utilizado**

[More about beta smoothing](https://stats.stackexchange.com/questions/47771/what-is-the-intuition-behind-beta-distribution/47782#47782) **Mais detalhes sobre o beta smoothing**